In [ ]:
import pandas as pd


file_path = 'email_process.csv'

# Read the CSV file
df = pd.read_csv(file_path)


df.dropna(subset=['Subject'], inplace=True) 
df['Date'] = pd.to_datetime(df['Date'], errors='coerce') 
df.drop_duplicates(subset=['Body'], inplace=True) 


df['Subject'] = df['Subject'].str.lower()
df['Body'] = df['Body'].str.lower()


df['Subject'] = df['Subject'].str.replace('[^a-zA-Z\s]', '', regex=True)
df['Body'] = df['Body'].str.replace('[^a-zA-Z\s]', '', regex=True)


df['From_Domain'] = df['From'].str.extract(r'@(\w+\.\w+)')
df['To_Domain'] = df['To'].str.extract(r'@(\w+\.\w+)')


df['Weekday'] = df['Date'].dt.day_name()
df['Hour'] = df['Date'].dt.hour

columns_to_drop = ['X-Folder', 'X-Origin', 'X-Filename']  
df.drop(columns=columns_to_drop, inplace=True)


df.to_csv('2_email_data.csv', index=False)


In [1]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Load the data
df = pd.read_csv('2_email_data.csv')

# For LDA, we need to convert the email bodies to a bag-of-words model
count_vect = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(df['Body'].values.astype('U'))

# Create and fit the LDA model
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(doc_term_matrix)

# Retrieve the words per topic
words = count_vect.get_feature_names()
for i, topic in enumerate(lda.components_):
    print(f"Top words for topic {i}:")
    print([words[i] for i in topic.argsort()[-10:]])
    print("\n")


MemoryError: Unable to allocate 27.7 GiB for an array with shape (227212,) and data type <U32759

In [2]:
import spacy

# Load Spacy's English tokenizer model
nlp = spacy.load('en_core_web_sm')

# Function to apply NER
def named_entity_recognition(text):
    doc = nlp(text)
    return [(X.text, X.label_) for X in doc.ents]

# Apply NER to the 'Body' of emails
df['named_entities'] = df['Body'].apply(named_entity_recognition)


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'float'>

In [5]:
import spacy

# Load the pre-trained NER model
nlp = spacy.load('en_core_web_sm')

# Function to extract entities
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Apply NER on the 'Body' column
df['entities'] = df['Body'].apply(extract_entities)

# View the entity extraction results
print(df['entities'].head())


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'float'>